In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset, load_dataset_scheme, Dataset
from lib.ehr.interface import Patients
from lib.ehr.concepts import CPRDDemographicVectorConfig, DemographicVectorConfig


In [3]:
import logging
import pprint

logging.root.level = logging.DEBUG
pp = pprint.PrettyPrinter(indent=4)


In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_FILE = f'{HOME}/GP/ehr-data/cprd-data/DUMMY_DATA.csv'
SOURCE_DIR = os.path.abspath("..")
cache_to_disk = 'cached_inteface/cprd_dummy'
use_cached =  'cached_inteface/cprd_dummy' # 'cached_inteface/m4inpatients_8000' #cache_to_disk        # False # 
use_cached =  False

##### Possible Interface Scheme Configurations

In [5]:
import json


interface_schem_options = load_dataset_scheme('CPRD').supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))


DEBUG:root:Constructing dx_cprd_ltc9809 (<class 'lib.ehr.coding_scheme.DxLTC9809FlatMedcodes'>) scheme
DEBUG:root:Constructing eth_cprd_16 (<class 'lib.ehr.coding_scheme.CPRDEthnicity16'>) scheme
DEBUG:root:Constructing cprd_gender (<class 'lib.ehr.coding_scheme.CPRDGender'>) scheme
DEBUG:root:Constructing cprd_imd_cat (<class 'lib.ehr.coding_scheme.CPRDIMDCategorical'>) scheme


{
    "dx": [
        "DxLTC9809FlatMedcodes",
        "DxLTC212FlatCodes"
    ],
    "ethnicity": [
        "CPRDEthnicity16",
        "CPRDEthnicity5"
    ],
    "gender": [
        "CPRDGender"
    ],
    "imd": [
        "CPRDIMDCategorical"
    ],
    "outcome": [
        "dx_cprd_ltc212",
        "dx_cprd_ltc9809"
    ]
}


In [6]:
cprd_interface_scheme_kw = dict(dx='DxLTC212FlatCodes',
                                outcome='dx_cprd_ltc212',
#                               outcome='dx_cprd_ltc9809',
                                ethnicity='CPRDEthnicity5')
# Demographic vector attributes
demographic_vector_conf = CPRDDemographicVectorConfig(age=True,
                                                          gender=True,
                                                          ethnicity=True,
                                                          imd=True)

In [ ]:
if use_cached:
    cprd_patients = Patients.load(use_cached)
    splits = cprd_patients.dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

else:
    with U.modified_environ(DATA_FILE=DATA_FILE), dask.config.set(scheduler='processes', num_workers=12):
        
        # Load dataset
        cprd_dataset = load_dataset('CPRD', sample=None)
        # Use training-split for fitting the outlier_remover and the scalers.
        splits = cprd_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')


        # Load interface
        cprd_patients = Patients(cprd_dataset, demographic_vector_conf,
                                **cprd_interface_scheme_kw).load_subjects(num_workers=12)

        # Cache to disk
        cprd_patients.save(cache_to_disk, overwrite=True)

In [ ]:
len(cprd_patients.subjects)

In [ ]:
from lib.ml import (ICENODE, ICENODEDimensions, OutpatientEmbeddingDimensions, 
                    Trainer, TrainerReporting, OptimizerConfig, WarmupConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric)

import jax.random as jrandom

In [ ]:
emb_dims = OutpatientEmbeddingDimensions(dx=30, demo=5)
dims = ICENODEDimensions(mem=15, emb=emb_dims)
key = jrandom.PRNGKey(0)

m = ICENODE(dims=dims, 
            schemes=cprd_patients.schemes,
            demographic_vector_config=cprd_patients.demographic_vector_config,
            key=key)

In [ ]:
splits = cprd_patients.random_splits([0.9, 0.95], 
                                    balanced='admissions')

trainer = Trainer(optimizer_config=OptimizerConfig(opt='adam', lr=1e-3),
                    reg_hyperparams=None,
                    epochs=80,
                    batch_size=128,
                    dx_loss='balanced_focal_softmax_bce')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)

loss_metric =  LossMetric(cprd_patients, 
                          dx_loss=('softmax_bce', 'balanced_focal_softmax_bce', 
                                   'balanced_focal_bce', 'allpairs_exp_rank', 'allpairs_hard_rank', 
                                   'allpairs_sigmoid_rank'))

metrics = [CodeAUC(cprd_patients), 
           AdmissionAUC(cprd_patients), 
           CodeGroupTopAlarmAccuracy(cprd_patients, n_partitions=5, 
                                     top_k_list=[3, 5, 10, 15, 20],
                                     train_split=splits[0]), 
           loss_metric]


reporting = TrainerReporting(output_dir='dx_icenode',
                             metrics=metrics,
                             console=True,
                             parameter_snapshots=True,
                             config_json=True)

In [ ]:
res = trainer(m, cprd_patients, 
              splits=splits,
              reporting=reporting,
              n_evals=1000,
              warmup_config=None, 
              continue_training=False)

In [ ]:
subject = cprd_patients.subjects[splits[0][0]]
admission = subject.admissions[0]
static_info = subject.static_info
len(static_info.demographic_vector(admission.admission_dates[0]))

In [ ]:
cprd_patients.scheme.demographic_vector_size(demographic_vector_conf)